##### PyCity Schools Analysis

- In this analysis, schools with higher budgets did not yield better test results. Schools with higher spending 645-675 dollars per student actually underperformed compared to schools with smaller budgets (585 dollars per student).

- Overall, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 58%).

- Charter schools out-performed the public district schools across all metrics. However, more analysis will be required to determine if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school.
---

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.
school_data_complete = pd.merge(student_data, school_data, how="left", on="school_name")
school_data_complete.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [2]:
# Calculate the total number of unique schools
school_count = len(school_data_complete.school_name.unique())
school_count


15

In [3]:
# Calculate the total number of students
student_count = school_data_complete.student_name.count()
student_count


39170

In [4]:
# Calculate the total budget
total_budget = sum(school_data_complete.budget.unique())
total_budget


24649428

In [5]:
# Calculate the average (mean) math score
average_math_score = school_data_complete.math_score.mean()
average_math_score


78.98537145774827

In [6]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete.reading_score.mean()
average_reading_score


81.87784018381414

In [7]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage


74.9808526933878

In [8]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage


85.80546336482001

In [9]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate


65.17232575950983

In [10]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({"Total Schools":[school_count],
                                 "Total Students":[student_count],
                                 "Total Budget":[total_budget],
                                 "Average Math Score":[average_math_score],
                                 "Average Reading Score":[average_reading_score],
                                 "% Passing Math":[passing_math_percentage*100],
                                 "% Passing Reading":[passing_reading_percentage*100],
                                 "% Overall Passing Rate":[overall_passing_rate*100]})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,7498.085269,8580.546336,6517.232576


## School Summary

In [11]:
# Use the code provided to select the type per school from school_data
school_summary = school_data_complete.groupby(['school_name'])
school_summary

school_names = school_data_complete.school_name.sort_values().unique()

school_types = school_data.set_index(["school_name"])["type"]
school_types

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Wilson High School        Charter
Cabrera High School       Charter
Bailey High School       District
Holden High School        Charter
Pena High School          Charter
Wright High School        Charter
Rodriguez High School    District
Johnson High School      District
Ford High School         District
Thomas High School        Charter
Name: type, dtype: object

In [12]:
# Calculate the total student count per school from school_data
per_school_counts = list(school_summary.student_name.count())
per_school_counts

[4976,
 1858,
 2949,
 2739,
 1468,
 4635,
 427,
 2917,
 4761,
 962,
 3999,
 1761,
 1635,
 2283,
 1800]

In [13]:
# Calculate the total school budget and per capita spending per school from school_data
per_school_budget = list(school_summary.budget.mean())
per_school_capita = [i/j for i,j in zip(per_school_budget,per_school_counts)]

In [14]:
# Calculate the average test scores per school from school_data_complete
per_school_math = list(school_summary.math_score.mean())
per_school_reading = list(school_summary.reading_score.mean())

In [15]:
# Calculate the number of students per school with math scores of 70 or higher from school_data_complete
students_passing_math = school_data_complete[school_data_complete['math_score'] >= 70]
school_students_passing_math = students_passing_math.groupby('school_name').size()
school_students_passing_math_df = school_students_passing_math.reset_index(name='count_passing_math')
print(school_students_passing_math_df)


              school_name  count_passing_math
0      Bailey High School                3318
1     Cabrera High School                1749
2    Figueroa High School                1946
3        Ford High School                1871
4     Griffin High School                1371
5   Hernandez High School                3094
6      Holden High School                 395
7       Huang High School                1916
8     Johnson High School                3145
9        Pena High School                 910
10  Rodriguez High School                2654
11    Shelton High School                1653
12     Thomas High School                1525
13     Wilson High School                2143
14     Wright High School                1680


In [16]:
# Calculate the number of students per school with reading scores of 70 or higher from school_data_complete
students_passing_reading = school_data_complete[school_data_complete['reading_score'] >= 70]
school_students_passing_reading = students_passing_reading.groupby('school_name').size()
school_students_passing_reading_df = school_students_passing_reading.reset_index(name='count_passing_reading')
print(school_students_passing_reading_df)


              school_name  count_passing_reading
0      Bailey High School                   4077
1     Cabrera High School                   1803
2    Figueroa High School                   2381
3        Ford High School                   2172
4     Griffin High School                   1426
5   Hernandez High School                   3748
6      Holden High School                    411
7       Huang High School                   2372
8     Johnson High School                   3867
9        Pena High School                    923
10  Rodriguez High School                   3208
11    Shelton High School                   1688
12     Thomas High School                   1591
13     Wilson High School                   2204
14     Wright High School                   1739


In [17]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()


In [18]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_students_passing_math / per_school_counts * 100
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100


In [19]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame({"School Names":school_names,
                                  "School Type":school_types,
                                  "Total Students":per_school_counts,
                                  "Total School Budget":per_school_budget,
                                  "Per Student Budget":per_school_capita,
                                  "Average Math Score":per_school_math,
                                  "Average Reading Score":per_school_reading,
                                  "% Passing Math":per_school_passing_math,
                                  "% Passing Reading":per_school_passing_reading,
                                  "Overall Passing Rate":overall_passing_rate})

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary


,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,,
Bailey High School,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Highest-Performing Schools (by % Overall Passing)

In [20]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(by='Overall Passing Rate', ascending=False).head(5).reset_index(drop=True)
top_schools.head(5)

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
1,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
2,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
3,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
4,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [21]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(by='Overall Passing Rate', ascending=True).head(5).reset_index(drop=True)
bottom_schools.head(5)


,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
2,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
4,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [22]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
    # Reset index to convert Series to Dataframe by using reset(index)
ninth_grade_math_scores = ninth_graders.groupby('school_name')['math_score'].mean().reset_index(name='ninth_grade_math_score')
tenth_grader_math_scores = tenth_graders.groupby('school_name')['math_score'].mean().reset_index(name='tenth_grade_math_score')
eleventh_grader_math_scores = eleventh_graders.groupby('school_name')['math_score'].mean().reset_index(name='eleventh_grade_math_score')
twelfth_grader_math_scores = twelfth_graders.groupby('school_name')['math_score'].mean().reset_index(name='twelfth_grade_math_score')

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = ninth_grade_math_scores.merge(tenth_grader_math_scores, on='school_name', how='outer').merge(eleventh_grader_math_scores, on='school_name', how='outer').merge(twelfth_grader_math_scores, on='school_name', how='outer')

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade


,school_name,ninth_grade_math_score,tenth_grade_math_score,eleventh_grade_math_score,twelfth_grade_math_score
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [23]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
    # Reset index to convert Series to Dataframe by using reset(index)
ninth_grade_reading_scores = ninth_graders.groupby('school_name')['reading_score'].mean().reset_index(name='ninth_grade_reading_score')
tenth_grader_reading_scores = tenth_graders.groupby('school_name')['reading_score'].mean().reset_index(name='tenth_grade_reading_score')
eleventh_grader_reading_scores = eleventh_graders.groupby('school_name')['reading_score'].mean().reset_index(name='eleventh_grade_reading_score')
twelfth_grader_reading_scores = twelfth_graders.groupby('school_name')['reading_score'].mean().reset_index(name='twelfth_grade_reading_score')

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = ninth_grade_reading_scores.merge(tenth_grader_reading_scores, on='school_name', how='outer') \
                                               .merge(eleventh_grader_reading_scores, on='school_name', how='outer') \
                                               .merge(twelfth_grader_reading_scores, on='school_name', how='outer')


# Minor data wrangling: Rename columns to "9th", "10th", "11th", "12th"
reading_scores_by_grade = reading_scores_by_grade.rename(columns={
    'ninth_grade_reading_score': '9th',
    'tenth_grade_reading_score': '10th',
    'eleventh_grade_reading_score': '11th',
    'twelfth_grade_reading_score': '12th'
})
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade


,school_name,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [24]:
# Establish the bins
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [25]:
# Create a copy of the school summary for later aggregations
school_spending_df = per_school_summary.copy()

school_spending_df

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,,
Bailey High School,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [26]:
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=labels)

school_spending_df

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,Spending Ranges (Per Student)
school_name,,,,,,,,,,,
Bailey High School,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,$585-630
Cabrera High School,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,<$585
Figueroa High School,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,$630-645
Ford High School,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,$630-645
Griffin High School,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,$585-630
Hernandez High School,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,$645-680
Holden High School,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,<$585
Huang High School,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680
Johnson High School,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,$645-680


In [27]:
#Calculate averages for the desired columns.
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Overall Passing Rate"].mean()

/var/folders/p7/44v4jhr91plb0p3405m1crv80000gn/T/ipykernel_38338/1103834571.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
/var/folders/p7/44v4jhr91plb0p3405m1crv80000gn/T/ipykernel_38338/1103834571.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
/var/folders/p7/44v4jhr91plb0p3405m1crv80000gn/T/ipykernel_38338/1103834571.py:4: FutureWarning: The default of observed=False

In [28]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({"Average Math Score": spending_math_scores,
                                        "Average Reading Score": spending_reading_scores,
                                        "% Passing Math": spending_passing_math,
                                        "% Passing Reading": spending_passing_reading,
                                        "% Overall Passing": overall_passing_spending})

# Display results
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

In [29]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


In [30]:
# Create a copy of the school summary for later aggregations
school_size_df = per_school_summary.copy()

In [31]:
# Use `pd.cut` on the per_school_counts Series from earlier to categorize school size based on the bins.
school_size_df["School Size"] = pd.cut(per_school_counts, size_bins, labels=labels)

# Convert School Size to a string
school_size_df["School Size"] = school_size_df["School Size"].astype(str)
school_size_df


,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,School Size
school_name,,,,,,,,,,,
Bailey High School,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,Large (2000-5000)
Cabrera High School,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,Medium (1000-2000)
Figueroa High School,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,Large (2000-5000)
Ford High School,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,Large (2000-5000)
Griffin High School,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,Medium (1000-2000)
Hernandez High School,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,Large (2000-5000)
Holden High School,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,Small (<1000)
Huang High School,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
Johnson High School,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,Large (2000-5000)


In [32]:
# Calculate averages for the desired columns.
size_math_scores = school_size_df.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = school_size_df.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = school_size_df.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = school_size_df.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = school_size_df.groupby(["School Size"])["Overall Passing Rate"].mean()


In [33]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
    # Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame({"Average Math Score": size_math_scores,
                                        "Average Reading Score": size_reading_scores,
                                        "% Passing Math": size_passing_math,
                                        "% Passing Reading": size_passing_reading,
                                        "% Overall Passing": size_overall_passing})

# Display results
size_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853


## Scores by School Type

In [34]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean().reset_index()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean().reset_index()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean().reset_index()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean().reset_index()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["Overall Passing Rate"].mean().reset_index()


In [35]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = average_math_score_by_type.merge(average_reading_score_by_type, on='School Type', how = 'outer') \
                                            .merge(average_percent_passing_math_by_type, on='School Type', how = 'outer') \
                                            .merge(average_percent_passing_reading_by_type, on='School Type', how = 'outer') \
                                            .merge(average_percent_overall_passing_by_type, on='School Type', how = 'outer')

# Display results
type_summary


,School Type,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Charter,83.473852,83.896421,93.620830,96.586489,90.432244
1,District,76.956733,80.966636,66.548453,80.799062,53.672208
